<a href="https://colab.research.google.com/github/Nour-Alhussien/AdaptiveScaledRandomizedSmoothing/blob/main/Github_Masking_Scaling_UNSW_Crossvalidation_Exper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/UNSW_Blanced')

In [ ]:
!pip install adversarial-robustness-toolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Load datasets
train = pd.read_csv('unsw_balanced_train_data.csv')
test = pd.read_csv('unsw_balanced_test_data.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Separate features and target
X_train = train.drop(columns=['Label']).values
y_train = train['Label'].values
X_test = test.drop(columns=['Label']).values
y_test = test['Label'].values

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# # Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(np.array(X_train), dtype=torch.float32)  # Ensure X_train is converted to numpy
y_train_tensor = torch.tensor(np.array(y_train), dtype=torch.long)
X_test_tensor = torch.tensor(np.array(X_test), dtype=torch.float32)
y_test_tensor = torch.tensor(np.array(y_test), dtype=torch.long)

print(f"Train features tensor shape: {X_train_tensor.shape}")
print(f"Train labels tensor shape: {y_train_tensor.shape}")
print(f"Train features tensor shape: {X_test_tensor.shape}")

Train features tensor shape: torch.Size([195874, 204])
Train labels tensor shape: torch.Size([195874])
Train features tensor shape: torch.Size([39176, 204])


In [ ]:
# Step 1: Find column names that start with 'cat__'
categorical_columns = [col for col in test.columns if col.startswith('cat__')]

# Step 2: Get their indices in the DataFrame
cat_idxs = [test.columns.get_loc(col) for col in categorical_columns]

print("Categorical Feature Indices:", cat_idxs)


Categorical Feature Indices: [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201]


In [ ]:
### Train a normal model to test it against the adversarail examples, to help us find the improvement in accuracy after the defense###
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from scipy.stats import norm
from statsmodels.stats.proportion import proportion_confint
from math import ceil
# Tabular Neural Network Definition
class TabularNN(nn.Module):
    def __init__(self, input_size, dropout_rate=0.25):
        super(TabularNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
### Train a normal model to test it against the adversarail examples, it help us find the improvement in accuracy after the defense###

import time

# Model setup
input_size = X_train_tensor.shape[1]
model = TabularNN(input_size=input_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader setup
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# File to save results
results_file = "evaluation_results.txt"

# Clear the results file if it exists
with open(results_file, "w") as f:
    f.write("Model Evaluation Results\n")
    f.write("=" * 50 + "\n")

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")
    with open(results_file, "a") as f:
        f.write(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}\n")



Epoch [1/10], Loss: 0.0259
Epoch [2/10], Loss: 0.0158
Epoch [3/10], Loss: 0.0155
Epoch [4/10], Loss: 0.0152
Epoch [5/10], Loss: 0.0150
Epoch [6/10], Loss: 0.0149
Epoch [7/10], Loss: 0.0148
Epoch [8/10], Loss: 0.0147
Epoch [9/10], Loss: 0.0146
Epoch [10/10], Loss: 0.0146


In [ ]:
def generate_adversarial_examples(attack, X_original, cat_idxs=None):

    X_adv = attack.generate(x=X_original)

    # Restore original categorical features to ensure they remain valid
    if cat_idxs and len(cat_idxs) > 0:
        X_adv[:, cat_idxs] = X_original[:, cat_idxs]

    return X_adv

X_test_np = X_test_tensor.cpu().numpy()

In [ ]:
from art.attacks.evasion import ProjectedGradientDescent, CarliniL2Method, ZooAttack, HopSkipJump, FastGradientMethod, DeepFool, SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np

# Ensure model and data are on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Create a PyTorchClassifier for ART
classifier = PyTorchClassifier(
     model=model,
     clip_values=(0, 1),
     loss=criterion,
     optimizer=optimizer,
     input_shape=(X_test_tensor.shape[1],),
     nb_classes=2,
     device_type="gpu" if torch.cuda.is_available() else "cpu",
 )

# Define attack parameters
zoo_attack = ZooAttack(
     classifier,
     confidence=0.8,  # Confidence of adversarial examples
     targeted=False,
     learning_rate=0.01,
     nb_parallel= 5,
     variable_h = 0.8,
     max_iter=40,  # Number of iterations
 )


cw_attack = CarliniL2Method(
     classifier=classifier,
     confidence=0.0,
     targeted=False,
     learning_rate=0.001,
     max_iter=10,  # Number of iterations
     binary_search_steps=5,
     initial_const=0.01,
     max_halving=3,
     max_doubling=3,
 )

pgd = ProjectedGradientDescent(estimator=classifier, eps=0.15, eps_step=0.01, max_iter=40) # norm is inf

deepfool = DeepFool(classifier, max_iter=10)

saliency = SaliencyMapMethod(classifier)

fgsm = FastGradientMethod(classifier, eps=0.15) # norm is inf

hsj = HopSkipJump(classifier, max_iter=64, max_eval=10000, init_eval=100, init_size=100)

# Convert data to numpy
X_test_np = X_test_tensor.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()


In [ ]:
# Generate PGD adversarial examples
print("Generating PGD adversarial examples...")
X_adv_pgd = generate_adversarial_examples(pgd, X_test_np, cat_idxs)
X_adv_pgd = pgd.generate(X_test_np)
np.savetxt("X_adv_pgd_unsw_M.txt", X_adv_pgd, delimiter=",")

print("Saved PGD adversarial examples to 'X_adv_pgd_unsw_M.txt'.")

# Generate FGSM adversarial examples
print("Generating FGSM adversarial examples...")
X_adv_fgsm = generate_adversarial_examples(fgsm, X_test_np, cat_idxs)
#X_adv_fgsm = fgsm.generate(X_test_np)
np.savetxt("X_adv_fgsm_unsw_M.txt", X_adv_fgsm, delimiter=",")

print("Saved FGSM adversarial examples to 'X_adv_fgsm_unsw_M.txt'.")

Generating PGD adversarial examples...


PGD - Batches:   0%|          | 0/1225 [00:00<?, ?it/s]

Saved PGD adversarial examples to 'X_adv_pgd_unsw_M.txt'.
Generating FGSM adversarial examples...
Saved FGSM adversarial examples to 'X_adv_fgsm_unsw_M.txt'.


In [ ]:
# Generate DeepFool adversarial examples
print("Generating DeepFool adversarial examples...")
X_adv_df = generate_adversarial_examples(deepfool, X_test_np, cat_idxs)
#X_adv_df = deepfool.generate(X_test_np)
np.savetxt("X_adv_df_unsw_M.txt", X_adv_df, delimiter=",")
print("Saved DeepFool adversarial examples to 'X_adv_df_unsw_M.txt'.")

# # Generate SaliencyMapMethod adversarial examples
# print("Generating SaliencyMapMethod adversarial examples...")
# X_adv_jsma = generate_adversarial_examples(saliency, X_test_np, cat_idxs)
# #X_adv_jsma = saliency.generate(X_test_np)
# np.savetxt("X_adv_jsma_unsw_M.txt", X_adv_jsma, delimiter=",")
# print("Saved JSMA adversarial examples to 'X_adv_jsma_unsw_M.txt'.")

Generating DeepFool adversarial examples...


DeepFool:   0%|          | 0/39176 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Generate HSJ adversarial examples
print("Generating HSJ adversarial examples...")
X_adv_hsj = generate_adversarial_examples(hsj, X_test_np, cat_idxs)
#X_adv_hsj = HopSkipJump.generate(X_test_np)
np.savetxt("X_adv_hsj_unsw_M.txt", X_adv_hsj, delimiter=",")
print("Saved HSJ adversarial examples to 'X_adv_hsj_unsw_M.txt'.")


Generating HSJ adversarial examples...


HopSkipJump:   0%|          | 0/39176 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.


Saved HSJ adversarial examples to 'X_adv_hsj_unsw.txt'.
Generating ZOO adversarial examples...


ZOO:   0%|          | 0/39176 [00:00<?, ?it/s]

In [ ]:
# Generate CW adversarial examples
print("Generating CW adversarial examples...")
X_adv_cw = generate_adversarial_examples(cw_attack, X_test_np, cat_idxs)
#X_adv_cw = cw_attack.generate(X_test_np)
#np.savetxt("X_adv_cw_agg_unsw_M.txt", X_adv_cw, delimiter=",")
np.savetxt("X_adv_cw_agg_unsw_M_lessaggr.txt", X_adv_cw, delimiter=",")

print("Saved CW adversarial examples to 'X_adv_cw_new_M.txt'.")

Generating CW adversarial examples...


C&W L_2:   0%|          | 0/39176 [00:00<?, ?it/s]

Saved CW adversarial examples to 'X_adv_cw_new_M.txt'.


In [ ]:
# Generate ZOO adversarial examples
print("Generating ZOO adversarial examples...")
X_adv_zoo = generate_adversarial_examples(zoo, X_test_np, cat_idxs)
#X_adv_zoo = zoo_attack.generate(X_test_np)
np.savetxt("X_adv_zoo_agg_unsw_M.txt", X_adv_zoo, delimiter=",")
print("Saved ZOO adversarial examples to 'X_adv_zoo_new_M.txt'.")

Generating ZOO adversarial examples...


ZOO:   0%|          | 0/39176 [00:00<?, ?it/s]

Saved ZOO adversarial examples to 'X_adv_zoo_new.txt'.


In [ ]:
import torch.nn.functional as F

# Clipping function
def clip(current, low_bound, up_bound):
    # Convert bounds to tensors
    low_bound = torch.tensor(low_bound, dtype=torch.float32, device=current.device).unsqueeze(0)  # Add batch dimension
    up_bound = torch.tensor(up_bound, dtype=torch.float32, device=current.device).unsqueeze(0)    # Add batch dimension

    # Clip values
    clipped = torch.max(torch.min(current, up_bound), low_bound)
    return clipped

In [ ]:
import torch

def lowProFool(x, model, weights, bounds, maxiters, alpha, lambda_, device):
    """
    LowProFool adversarial attack implementation with weighted loss.

    Args:
        x (torch.Tensor): Input sample.
        model (torch.nn.Module): Target model.
        weights (list or np.array): Feature importance weights.
        bounds (tuple): Feature bounds as (min, max).
        maxiters (int): Maximum number of iterations.
        alpha (float): Step size for gradient update.
        lambda_ (float): Regularization weight for L2 loss.
        device (torch.device): Device to run computations on.

    Returns:
        orig_pred (int): Original prediction class.
        output_pred (int): Adversarial prediction class.
        best_pert_x (np.array): Adversarial example.
    """
    # Initialize inputs and parameters
    x = x.unsqueeze(0).to(device).float()  # Ensure input is float32
    x.requires_grad = True
    r = torch.tensor(1e-4 * torch.ones_like(x), device=device, requires_grad=True)  # Perturbation
    v = torch.tensor(weights, device=device).float()  # Feature importance weights

    # Initial prediction
    logits = model(x + r)
    orig_pred = logits.argmax(dim=1).item()

    # Define target class (opposite of original prediction)
    target_pred = 1 - orig_pred
    target = torch.tensor([target_pred], device=device).long()

    # Define loss functions
    criterion = torch.nn.CrossEntropyLoss()
    l2 = lambda v, r: torch.sqrt(torch.sum((v * r) ** 2))  # Weighted L2 norm

    best_norm_weighted = float('inf')
    best_pert_x = x.clone()

    # Iterative attack
    for loop_i in range(maxiters):
        if r.grad is not None:
            r.grad.zero_()

        # Compute logits and losses
        logits = model(x + r)
        loss_1 = criterion(logits, target)  # Classification loss
        loss_2 = l2(v, r)  # Regularization loss
        loss = loss_1 + lambda_ * loss_2  # Total loss

        # Backpropagate the loss
        loss.backward()

        # Gradient update for perturbation
        with torch.no_grad():
            r -= alpha * r.grad  # Update perturbation

        # Clamp the perturbed sample within the bounds
        xprime = x + r
        bounds_min = torch.tensor(bounds[0], device=device).float()
        bounds_max = torch.tensor(bounds[1], device=device).float()
        xprime = torch.clamp(xprime, bounds_min, bounds_max)

        # Evaluate adversarial prediction
        logits = model(xprime)
        output_pred = logits.argmax(dim=1).item()

        # Update best adversarial example based on weighted L2 norm
        weighted_norm = torch.sum(torch.abs(r * v)).item()
        if output_pred != orig_pred and weighted_norm < best_norm_weighted:
            best_norm_weighted = weighted_norm
            best_pert_x = xprime.clone()

    # Ensure the final adversarial example is clamped
    best_pert_x = torch.clamp(best_pert_x, bounds_min, bounds_max)
    logits = model(best_pert_x)
    output_pred = logits.argmax(dim=1).item()

    return orig_pred, output_pred, best_pert_x.squeeze(0).detach().cpu().numpy()


In [ ]:
import numpy as np
import torch
from torch.autograd import Variable
from scipy.stats import pearsonr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming model and data (X_train_np, X_test_np) are already loaded and preprocessed.

# Adjust feature importance computation to avoid constant features
feature_importance_weights = []
for i in range(X_train.shape[1]):
    if np.all(X_train[:, i] == X_train[0, i]):  # Check if the feature is constant
        feature_importance_weights.append(0.0)  # Assign zero weight to constant features
    else:
        correlation, _ = pearsonr(X_train[:, i], y_train)
        feature_importance_weights.append(abs(correlation))

feature_importance_weights = np.array(feature_importance_weights, dtype=np.float32)

# Define feature bounds based on the training data
feature_min = X_train.min(axis=0)
feature_max = X_train.max(axis=0)
feature_bounds = (feature_min, feature_max)

# Parameters for LowProFool attack
#max_iters = 50
max_iters = 20 #less aggressive
alpha = 0.005
lambda_ = 0.5

# Store adversarial examples
adversarial_examples = []
adversarial_accuracy = 0
num_samples = X_test.shape[0]  # Number of test samples

model.eval()  # Set the model to evaluation mode
for i in range(num_samples):  # Iterate through each test sample
    #print(f"Processing test sample {i + 1}/{num_samples}...")

    # Access each test sample from the numpy array
    x_test_sample = X_test[i]

    # Convert the test sample to torch.Tensor
    x_test_sample_tensor = torch.tensor(x_test_sample, dtype=torch.float32).to(device)

    # Generate adversarial example using LowProFool attack
    orig_pred, adv_pred, adv_example = lowProFool(
        x_test_sample_tensor, model, feature_importance_weights, feature_bounds, max_iters, alpha, lambda_, device
    )

    # Store the generated adversarial example
    adversarial_examples.append(adv_example)

    # Check if the original prediction and adversarial prediction are the same
    if orig_pred == adv_pred:
        adversarial_accuracy += 1

# Convert adversarial examples to a numpy array
adversarial_examples = np.array(adversarial_examples)

# Save all adversarial examples to a single file
#np.savetxt("X_adv_lowprofool_M.csv", adversarial_examples, delimiter=",")
np.savetxt("X_adv_lowprofool_M_lessaggr.csv", adversarial_examples, delimiter=",")


# Calculate adversarial accuracy
adversarial_accuracy /= num_samples
print(f'Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%')


/tmp/ipython-input-409100749.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r = torch.tensor(1e-4 * torch.ones_like(x), device=device, requires_grad=True)  # Perturbation


Adversarial Accuracy: 49.90%


In [ ]:
# Step 1: Find column names that start with 'cat__'
categorical_columns = [col for col in test.columns if col.startswith('cat__')]

# Step 2: Add the two specific features if they exist in the DataFrame
extra_features = ['is_sm_ips_ports', 'is_ftp_login']
for feat in extra_features:
    if feat in test.columns and feat not in categorical_columns:
        categorical_columns.append(feat)

# Step 3: Get their indices in the DataFrame
cat_idxs = [test.columns.get_loc(col) for col in categorical_columns]

print("Categorical Feature Indices:", cat_idxs)


Categorical Feature Indices: [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 202]


In [ ]:
import numpy as np
import pandas as pd
import os

# Example mapping of attack name to file path
adversarial_loaders = {
    "PGD": "X_adv_pgd_unsw2.txt",
    "JSMA": "X_adv_jsma_unsw.txt",
    "DeepFool": "X_adv_df_unsw.txt",
    "FGSM": "X_adv_fgsm_unsw2.txt",
    "ZOO": "X_adv_zoo_agg_unsw.txt",
    "CW": "X_adv_cw_agg_unsw2.txt",
    "LPF": "X_adv_lowprofool_lessagg_unsw.csv",
    "HSJ": "X_adv_hsj_unsw.txt",
}


# Process each adversarial file
for attack_name, file_path in adversarial_loaders.items():
    # Load adversarial examples (support both .txt and .csv)
    if file_path.endswith(".csv"):
        X_adv = pd.read_csv(file_path, header=None).values
    else:
        X_adv = np.loadtxt(file_path, delimiter=",")

    # Restore categorical features from X_test_np
    X_adv[:, cat_idxs] = X_test_np[:, cat_idxs]

    # Save restored version
    save_path = f"X_adv_{attack_name.lower()}_unsw_restored_M.csv"
    np.savetxt(save_path, X_adv, delimiter=",")

    print(f"{attack_name} _unsw_restored and saved to {save_path}")


NameError: name 'cat_idxs' is not defined

In [ ]:
# Testing on adversarial examples
adversarial_loaders = {
    "PGD": "X_adv_pgd_unsw_restored_M.csv",
    "JSMA": "X_adv_jsma_unsw_restored_M.csv",
    "DeepFool": "X_adv_deepfool_unsw_restored_M.csv",
    "FGSM": "X_adv_fgsm_unsw_restored_M.csv",
    "ZOO": "X_adv_zoo_unsw_restored_M.csv",
    "CW": "X_adv_cw_unsw_restored_M.csv",
    "LPF": "X_adv_lpf_unsw_restored_M.csv",
    "HSJ": "X_adv_hsj_unsw_restored_M.csv",

}

# Testing on clean test data
model.eval()
with torch.no_grad():
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)
    test_outputs = model(X_test_tensor)
    test_predictions = torch.argmax(test_outputs, axis=1)
    clean_accuracy = accuracy_score(y_test_tensor.cpu().numpy(), test_predictions.cpu().numpy())
    print(f"Clean test accuracy: {clean_accuracy:.4f}")
    with open(results_file, "a") as f:
        f.write(f"\nClean test accuracy: {clean_accuracy:.4f}\n")

# Evaluate on adversarial examples
for attack_name, file_path in adversarial_loaders.items():
    # Load adversarial data
    X_adv = np.loadtxt(file_path, delimiter=",")  # Load the CSV as a NumPy array
    X_adv_tensor = torch.tensor(X_adv, dtype=torch.float32).to(device)
    # to solve the mismatch size between x_adv and  y_true

    # Record evaluation time
    start_time = time.time()
    with torch.no_grad():
        adv_outputs = model(X_adv_tensor)
        adv_predictions = torch.argmax(adv_outputs, axis=1)
    end_time = time.time()

    # Compute adversarial accuracy
    adv_accuracy = accuracy_score(y_test_tensor.cpu().numpy(), adv_predictions.cpu().numpy())
    eval_time = end_time - start_time

    # Print and save results
    print(f"{attack_name} adversarial accuracy: {adv_accuracy:.4f}")
    print(f"Time taken for {attack_name} evaluation: {eval_time:.4f} seconds")
    with open(results_file, "a") as f:
        f.write(f"{attack_name} adversarial accuracy: {adv_accuracy:.4f}\n")
        f.write(f"Time taken for {attack_name} evaluation: {eval_time:.4f} seconds\n")


Clean test accuracy: 0.9969
PGD adversarial accuracy: 0.1220
Time taken for PGD evaluation: 0.0008 seconds
PGDnorm2 adversarial accuracy: 0.9943
Time taken for PGDnorm2 evaluation: 0.0005 seconds
JSMA adversarial accuracy: 0.5041
Time taken for JSMA evaluation: 0.0005 seconds
DeepFool adversarial accuracy: 0.9793
Time taken for DeepFool evaluation: 0.0005 seconds
FGSM adversarial accuracy: 0.4371
Time taken for FGSM evaluation: 0.0005 seconds
FGSMnorm2 adversarial accuracy: 0.9956
Time taken for FGSMnorm2 evaluation: 0.0005 seconds
ZOO adversarial accuracy: 0.9969
Time taken for ZOO evaluation: 0.0005 seconds
CW adversarial accuracy: 0.7068
Time taken for CW evaluation: 0.0004 seconds
CWlessaggre adversarial accuracy: 0.4206
Time taken for CWlessaggre evaluation: 0.0005 seconds
LPF adversarial accuracy: 0.5001
Time taken for LPF evaluation: 0.0004 seconds
LPFlessaggre adversarial accuracy: 0.5001
Time taken for LPFlessaggre evaluation: 0.0005 seconds
HSJ adversarial accuracy: 0.9969
Ti

In [ ]:
###########ADAPTIVE MODEL##############

###UPDATING PARAMETERS adding noise based on model entropy, high entropy means less confident model-> add less noise and vice versa###
##Update the raduis calculation using Margin-based sampling
##Use daptive scaling and proxy metric to calculate margin

import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from scipy.stats import norm
from statsmodels.stats.proportion import proportion_confint
from math import ceil
import matplotlib.pyplot as plt
import pandas as pd


class TabularNN(nn.Module):
    def __init__(self, input_size, dropout_rate=0.25):
        super(TabularNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x


class AdaptiveSmoothEntropy:
    ABSTAIN = -1

    def __init__(self, base_classifier, num_classes, initial_sigma=1):
        self.base_classifier = base_classifier
        self.num_classes = num_classes
        self.sigma = initial_sigma
        self.margin_logs = []

    def train_model(self, train_loader, test_loader, num_epochs=30):
        optimizer = optim.AdamW(self.base_classifier.parameters(), lr=0.001, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        for epoch in range(num_epochs):
            running_loss = 0.0
            self.base_classifier.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs.requires_grad = True
                optimizer.zero_grad()

                if epoch < num_epochs // 2:
                    outputs_clean = self.base_classifier(inputs)
                    loss = nn.CrossEntropyLoss()(outputs_clean, labels)
                else:
                    clean_inputs, noisy_inputs = self.mix_clean_and_noisy(inputs)
                    outputs_clean = self.base_classifier(clean_inputs)
                    outputs_noisy = self.base_classifier(noisy_inputs)
                    loss_clean = nn.CrossEntropyLoss()(outputs_clean, labels)
                    loss_noisy = nn.CrossEntropyLoss()(outputs_noisy, labels)
                    loss = loss_clean + loss_noisy

                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            scheduler.step()
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

        self.evaluate(test_loader)

    def mix_clean_and_noisy(self, inputs):
        clean_inputs = inputs.clone()
        with torch.no_grad():
            probs = torch.softmax(self.base_classifier(clean_inputs), dim=1)
            entropies = -torch.sum(probs * torch.log(probs + 1e-8), dim=1)
        adaptive_noise_scale = 1 - (entropies / np.log(self.num_classes))
        noise = torch.randn_like(inputs) * self.sigma * adaptive_noise_scale.unsqueeze(1)
        noisy_inputs = inputs + noise
        return clean_inputs, noisy_inputs

    def evaluate(self, loader):
        self.base_classifier.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.base_classifier(inputs)
                predicted = torch.argmax(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Accuracy on clean data: {accuracy:.2f}%")
        return accuracy

    def _proxy_margin(self, x):
        with torch.no_grad():
            probs = torch.softmax(self.base_classifier(x), dim=1).squeeze()
        top2 = torch.topk(probs, 2).values
        return (top2[0] - top2[1]).item(), top2[0].item()

    def _adaptive_sample_size(self, margin, confidence):
        scale = max(1.0, (1.0 - margin) * (1.0 - confidence)) * 2
        return int(ceil(1000 * scale))

    def certify(self, x, n0, n_base, alpha, batch_size):
        self.base_classifier.eval()
        try:
            counts_selection = self._sample_noise(x, n0, batch_size)
            cAHat = counts_selection.argmax().item()

            margin, confidence = self._proxy_margin(x)
            n = self._adaptive_sample_size(margin, confidence)

            counts_estimation = self._sample_noise(x, n, batch_size)
            nA = counts_estimation[cAHat].item()
            pABar = self._lower_confidence_bound(nA, n, alpha)

            if pABar < 0.5:
                predicted_class = AdaptiveSmoothEntropy.ABSTAIN
                radius = 0.0
            else:
                predicted_class = cAHat
                radius = self.sigma * norm.ppf(pABar)

            return predicted_class, radius
        except Exception as e:
            print(f"Error: {e}")
            return AdaptiveSmoothEntropy.ABSTAIN, 0.0

    def _sample_noise(self, x, num, batch_size):
        counts = np.zeros(self.num_classes, dtype=int)
        with torch.no_grad():
            for _ in range(ceil(num / batch_size)):
                this_batch_size = min(batch_size, num)
                num -= this_batch_size
                batch = x.repeat((this_batch_size, 1))
                noise = torch.randn_like(batch) * self.sigma
                predictions = self.base_classifier(batch + noise).argmax(1)
                counts += np.bincount(predictions.cpu().numpy(), minlength=self.num_classes)
        return counts

    def _lower_confidence_bound(self, NA, N, alpha):
        return proportion_confint(NA, N, alpha=2 * alpha, method="beta")[0]

    def test_on_adversarial(self, X_adv_test, y_test, method="margin", n0=100, n=1000, alpha=0.001, batch_size=64):
        total = len(X_adv_test)
        correct = 0
        certified_correct = 0

        with torch.no_grad():
            for i in range(total):
                x_adv_sample = torch.tensor(X_adv_test[i]).unsqueeze(0).to(device)
                y_true = y_test[i].item()

                certified_class, certified_radius = self.certify(
                    x_adv_sample, n0=n0, n_base=n, alpha=alpha, batch_size=batch_size)

                outputs = self.base_classifier(x_adv_sample)
                predicted = torch.argmax(outputs, dim=1).item()

                if predicted == y_true:
                    correct += 1
                if certified_class == y_true:
                    certified_correct += 1

        accuracy = correct / total * 100
        certified_accuracy = certified_correct / total * 100
        print(f"{method.capitalize()} Accuracy on adversarial examples: {accuracy:.2f}%")
        print(f"{method.capitalize()} Certified accuracy: {certified_accuracy:.2f}%")
        return accuracy, certified_accuracy




In [ ]:
###########ADAPTIVE MODEL##############
def train_and_evaluate_on_adversarial_folds(
    base_model_class,
    adaptive_model_class,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    adversarial_loaders,
    num_classes,
    initial_sigma,
    batch_size=64,
    num_epochs=30,
    folds=5,
    output_file="upd_adapted_adversarial_evaluation_results.txt",
):
    import torch
    from torch.utils.data import DataLoader, TensorDataset, Subset
    from sklearn.model_selection import KFold
    import numpy as np
    from collections import defaultdict
    import os
    import time

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    # Convert testing dataset to PyTorch DataLoader
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)

    # Prepare output file
    with open(output_file, "w") as f:
        f.write("===== Adversarial Evaluation Results =====\n")

    # Initialize and train the base model
    print("Initializing models...")
    base_model = base_model_class(input_size=X_train_tensor.shape[1], dropout_rate=0.5).to(device)
    adaptive_model = adaptive_model_class(
        base_classifier=base_model,
        num_classes=num_classes,
        initial_sigma=initial_sigma,
    )

    # Training on full training dataset
    print("Training AdaptiveSmoothEntropy model...")
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    adaptive_model.train_model(train_loader, test_loader, num_epochs=num_epochs)

    # Evaluate on clean testing data
    print("Evaluating AdaptiveSmoothEntropy model on clean testing data...")
    clean_accuracy = adaptive_model.evaluate(test_loader)
    print(f"Clean Test Accuracy: {clean_accuracy:.2f}%")
    with open(output_file, "a") as f:
        f.write(f"\nClean Test Accuracy: {clean_accuracy:.2f}%\n")

    # Evaluate adversarial examples using folds
    fold_results = []
    timing_results = []
    adversarial_accuracies_summary = defaultdict(list)

    for fold, (_, test_idx) in enumerate(kf.split(test_dataset)):
        print(f"\n===== Fold {fold + 1}/{folds} =====")

        val_subset = Subset(test_dataset, test_idx)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        y_val = torch.stack([val_subset[i][1] for i in range(len(val_subset))]).cpu().numpy()

        fold_adversarial_accuracies = {}
        fold_timing = {}

        for attack_name, loader in adversarial_loaders.items():
            print(f"\nEvaluating {attack_name} adversarial examples on Fold {fold + 1}...")

            # Load adversarial examples corresponding to the fold
            X_adv_loaded = np.loadtxt(loader, delimiter=",")
            X_adv_tensor = torch.tensor(X_adv_loaded[test_idx], dtype=torch.float32).to(device)

            # Time evaluation
            start_time = time.time()
            # Test on adversarial examples
            accuracy, certified_accuracy = adaptive_model.test_on_adversarial(
                X_adv_tensor,
                y_val,
                method="margin",
                n0=100,
                n=1000,
                alpha=0.001,
                batch_size=batch_size,
            )
            elapsed_time = time.time() - start_time

            fold_adversarial_accuracies[attack_name] = {
                "accuracy": accuracy,
                "certified_accuracy": certified_accuracy,
            }
            fold_timing[attack_name] = elapsed_time
            adversarial_accuracies_summary[attack_name].append((accuracy, certified_accuracy))

        # Log fold results
        with open(output_file, "a") as f:
            f.write(f"\nFold {fold + 1}:\n")
            f.write(f"  Clean Accuracy: {clean_accuracy:.2f}%\n")
            for attack_name, metrics in fold_adversarial_accuracies.items():
                f.write(f"  {attack_name}: Accuracy = {metrics['accuracy']:.2f}%, Certified Accuracy = {metrics['certified_accuracy']:.2f}%\n")
                f.write(f"  {attack_name}: Time = {fold_timing[attack_name]:.2f} seconds\n")

        fold_results.append({
            "fold": fold + 1,
            "adversarial_accuracies": fold_adversarial_accuracies,
        })
        timing_results.append(fold_timing)

    avg_timing = {
        attack_name: np.mean([timing[attack_name] for timing in timing_results])
        for attack_name in adversarial_loaders.keys()
    }

    # Final Summary
    print("\n===== Adversarial Evaluation Summary =====")
    with open(output_file, "a") as f:
        f.write("\n===== Adversarial Evaluation Summary =====\n")
        for attack_name, metrics in adversarial_accuracies_summary.items():
            avg_accuracy = np.mean([m[0] for m in metrics])
            avg_certified_accuracy = np.mean([m[1] for m in metrics])
            print(f"{attack_name}: Average Accuracy = {avg_accuracy:.2f}%, Average Certified Accuracy = {avg_certified_accuracy:.2f}%")
            f.write(f"{attack_name}: Average Accuracy = {avg_accuracy:.2f}%, Certified Accuracy = {avg_certified_accuracy:.2f}%\n")

        # Log average timing
        print("\n===== Average Timing Summary =====")
        f.write("\n===== Average Timing Summary =====\n")
        for attack_name, avg_time in avg_timing.items():
            print(f"  {attack_name}: Average Time = {avg_time:.2f} seconds")
            f.write(f"  {attack_name}: Average Time = {avg_time:.2f} seconds\n")


    return fold_results


In [ ]:
X_test_np = X_test_tensor.cpu().numpy()
np.savetxt("X_test_clean.txt", X_test_np, delimiter=",")

In [ ]:
###########ADAPTIVE MODEL##############
adversarial_loaders = {
    "PGD": "X_adv_pgd_unsw_restored_M.csv",
    "JSMA": "X_adv_jsma_unsw_restored_M.csv",
    "DeepFool": "X_adv_deepfool_unsw_restored_M.csv",
    "FGSM": "X_adv_fgsm_unsw_restored_M.csv",
    "ZOO": "X_adv_zoo_unsw_restored_M.csv",
    "CW": "X_adv_cw_unsw_restored_M.csv",
    "LPF": "X_adv_lpf_unsw_restored_M.csv",
    "HSJ": "X_adv_hsj_unsw_restored_M.csv",
}

results = train_and_evaluate_on_adversarial_folds(
    base_model_class=TabularNN,
    adaptive_model_class=AdaptiveSmoothEntropy,
    X_train_tensor=X_train_tensor,
    y_train_tensor=y_train_tensor,
    X_test_tensor=X_test_tensor,
    y_test_tensor=y_test_tensor,
    adversarial_loaders=adversarial_loaders,
    num_classes=2,
    initial_sigma=0.5,
    batch_size=64,
    num_epochs=30,
    folds=5,
    output_file="results/masking_less_unsw_adaptive_smoothing.txt",
)


Initializing models...
Training AdaptiveSmoothEntropy model...
Epoch 1/30, Loss: 0.0230
Epoch 2/30, Loss: 0.0166
Epoch 3/30, Loss: 0.0162
Epoch 4/30, Loss: 0.0158
Epoch 5/30, Loss: 0.0153
Epoch 6/30, Loss: 0.0154
Epoch 7/30, Loss: 0.0152
Epoch 8/30, Loss: 0.0150
Epoch 9/30, Loss: 0.0149
Epoch 10/30, Loss: 0.0149
Epoch 11/30, Loss: 0.0145
Epoch 12/30, Loss: 0.0146
Epoch 13/30, Loss: 0.0144
Epoch 14/30, Loss: 0.0143
Epoch 15/30, Loss: 0.0145
Epoch 16/30, Loss: 0.3929
Epoch 17/30, Loss: 0.3085
Epoch 18/30, Loss: 0.2915
Epoch 19/30, Loss: 0.2867
Epoch 20/30, Loss: 0.2845
Epoch 21/30, Loss: 0.2831
Epoch 22/30, Loss: 0.2806
Epoch 23/30, Loss: 0.2783
Epoch 24/30, Loss: 0.2780
Epoch 25/30, Loss: 0.2766
Epoch 26/30, Loss: 0.2772
Epoch 27/30, Loss: 0.2762
Epoch 28/30, Loss: 0.2766
Epoch 29/30, Loss: 0.2750
Epoch 30/30, Loss: 0.2718
Accuracy on clean data: 99.29%
Evaluating AdaptiveSmoothEntropy model on clean testing data...
Accuracy on clean data: 99.29%
Clean Test Accuracy: 99.29%

===== Fold 

/tmp/ipython-input-2938742028.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_adv_sample = torch.tensor(X_adv_test[i]).unsqueeze(0).to(device)


Margin Accuracy on adversarial examples: 98.85%
Margin Certified accuracy: 93.06%

Evaluating PGDnorm2 adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 99.20%
Margin Certified accuracy: 98.93%

Evaluating FGSM adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 98.51%
Margin Certified accuracy: 96.82%

Evaluating FGSMnorm2 adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 99.21%
Margin Certified accuracy: 98.92%

Evaluating CW adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 85.67%
Margin Certified accuracy: 95.60%

Evaluating CWlessaggre adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 63.00%
Margin Certified accuracy: 61.94%

Evaluating LPF adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 78.70%
Margin Certified accuracy: 87.67%

Evaluating LPFlessaggre adversarial examples on Fold 1...
Margin Accuracy on adversarial examples: 78.70%


In [ ]:
###code to compare with the original randomized smoothing###

import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import norm
from statsmodels.stats.proportion import proportion_confint
import numpy as np
from math import ceil

# Define your Tabular Neural Network
class TabularNN(nn.Module):
    def __init__(self, input_size, dropout_rate=0.25):
        super(TabularNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)  # Output 2 logits for binary classification
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)  # Output logits
        return x

# Original Randomized Smoothing (certification method)
class Smooth:
    ABSTAIN = -1

    def __init__(self, base_classifier: nn.Module, num_classes: int, sigma: float):
        self.base_classifier = base_classifier
        self.num_classes = num_classes
        self.sigma = sigma

    def certify(self, x: torch.tensor, n0: int, n: int, alpha: float, batch_size: int):
        self.base_classifier.eval()
        counts_selection = self._sample_noise(x, n0, batch_size)
        cAHat = counts_selection.argmax().item()
        counts_estimation = self._sample_noise(x, n, batch_size)
        nA = counts_estimation[cAHat].item()
        pABar = self._lower_confidence_bound(nA, n, alpha)
        if pABar < 0.5:
            predicted_class = Smooth.ABSTAIN
            radius = 0.0
        else:
            predicted_class = cAHat
            radius = self.sigma * norm.ppf(pABar)

        # Save results to a text file
        file_path = "/content/drive/MyDrive/UNSW_Blanced/original_smoothing_results.txt"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)  # Ensure the directory exists
        with open(file_path, "a") as file:
            file.write(f"Predicted Class: {predicted_class}, Radius: {radius}, Abstain: {predicted_class == Smooth.ABSTAIN}\n")

        return predicted_class, radius

    def _sample_noise(self, x: torch.tensor, num: int, batch_size: int):
        with torch.no_grad():
            counts = np.zeros(self.num_classes, dtype=int)
            for _ in range(ceil(num / batch_size)):
                this_batch_size = min(batch_size, num)
                num -= this_batch_size
                batch = x.repeat((this_batch_size, 1))
                noise = torch.randn_like(batch) * self.sigma
                predictions = self.base_classifier(batch + noise).argmax(1)
                counts += np.bincount(predictions.cpu().numpy(), minlength=self.num_classes)
            return counts

    def _lower_confidence_bound(self, NA: int, N: int, alpha: float) -> float:
        return proportion_confint(NA, N, alpha=2 * alpha, method="beta")[0]

# Training loop with noise
def train_model(model, train_loader, criterion, optimizer, noise_sd, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            noisy_inputs = inputs + torch.randn_like(inputs) * noise_sd
            outputs = model(noisy_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

# Evaluation function
def evaluate_model(model, test_loader, noise_sd):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            noisy_inputs = inputs + torch.randn_like(inputs) * noise_sd
            outputs = model(noisy_inputs)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
        accuracy = 100 * correct / total # Calculate accuracy
        print(f"Test Accuracy: {100 * correct / total:.2f}%")
        return accuracy # Return the calculated accuracy


def test_on_adversarial(self, X_adv_test, y_test, n0=100, n=1000, alpha=0.001, batch_size=64):
    """Test model on adversarial examples with certification."""
    total = len(X_adv_test)
    correct = 0
    certified_correct = 0

    with torch.no_grad():
        for i in range(total):
            x_adv_sample = torch.tensor(X_adv_test[i]).unsqueeze(0).to(device)
            y_true = y_test[i].item()

            certified_class, certified_radius = self.certify(
                x_adv_sample, n0=n0, n=n, alpha=alpha, batch_size=batch_size
            )

            outputs = self.base_classifier(x_adv_sample)
            predicted = torch.argmax(outputs, dim=1).item()

            if predicted == y_true:
                correct += 1
            if certified_class == y_true:
                certified_correct += 1

    accuracy = correct / total * 100
    certified_accuracy = certified_correct / total * 100
    print(f" Accuracy on adversarial examples: {accuracy:.2f}%")
    print(f" Certified accuracy: {certified_accuracy:.2f}%")
    return accuracy, certified_accuracy



In [ ]:
def train_and_evaluate_on_adversarial_folds(
    base_model_class,
    adaptive_model_class,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    adversarial_loaders,
    num_classes,
    initial_sigma,
    batch_size=64,
    num_epochs=30,
    folds=5,
    output_file="unsw_adversarial_evaluation_results_originalSmoothing.txt",
):
    import torch
    from torch.utils.data import DataLoader, TensorDataset, Subset
    from sklearn.model_selection import KFold
    import numpy as np
    from collections import defaultdict
    import time

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert testing dataset to PyTorch DataLoader
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)

    # Prepare output file
    with open(output_file, "w") as f:
        f.write("===== Adversarial Evaluation Results =====\n")

    # Initialize and train the base model
    print("Initializing models...")
    base_model = base_model_class(input_size=X_train_tensor.shape[1], dropout_rate=0.5).to(device)
    adaptive_model = adaptive_model_class(
        base_classifier=base_model,
        num_classes=num_classes,
        sigma=initial_sigma
    )

    # Training on full training dataset
    print("Training OriginalSmooth model...")
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define criterion and optimizer here:
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(base_model.parameters())
    noise_sd = 0.5

    train_model(base_model, train_loader, criterion, optimizer, noise_sd, num_epochs)

    # Evaluate on clean testing data
    print("Evaluating OriginalSmooth model on clean testing data...")
    clean_accuracy = evaluate_model(base_model, test_loader, noise_sd=noise_sd)
    print(f"Clean Test Accuracy: {clean_accuracy:.2f}%")
    with open(output_file, "a") as f:
        f.write(f"\nClean Test Accuracy: {clean_accuracy:.2f}%\n")

    # Evaluate adversarial examples using folds
    fold_results = []
    timing_results = []
    adversarial_accuracies_summary = defaultdict(list)

    for fold, (_, test_idx) in enumerate(kf.split(test_dataset)):
        print(f"\n===== Fold {fold + 1}/{folds} =====")

        val_subset = Subset(test_dataset, test_idx)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        y_val = torch.stack([val_subset[i][1] for i in range(len(val_subset))]).cpu().numpy()

        fold_adversarial_accuracies = {}
        fold_timing = {}
        for attack_name, loader in adversarial_loaders.items():
            print(f"\nEvaluating {attack_name} adversarial examples on Fold {fold + 1}...")

            # Load adversarial examples corresponding to the fold
            X_adv_loaded = np.loadtxt(loader, delimiter=",")
            X_adv_tensor = torch.tensor(X_adv_loaded[test_idx], dtype=torch.float32).to(device)

             # Time evaluation
            start_time = time.time()
            # Test on adversarial examples
            accuracy, certified_accuracy = test_on_adversarial(
                adaptive_model,
                X_adv_tensor,
                y_val,
                n0=100,
                n=1000,
                alpha=0.001,
                batch_size=batch_size,
            )
            elapsed_time = time.time() - start_time

            fold_adversarial_accuracies[attack_name] = {
                "accuracy": accuracy,
                "certified_accuracy": certified_accuracy,
            }
            fold_timing[attack_name] = elapsed_time
            adversarial_accuracies_summary[attack_name].append((accuracy, certified_accuracy))

        # Log fold results
        with open(output_file, "a") as f:
            f.write(f"\nFold {fold + 1}:\n")
            f.write(f"  Clean Accuracy: {clean_accuracy:.2f}%\n")
            for attack_name, metrics in fold_adversarial_accuracies.items():
                f.write(f"  {attack_name}: Accuracy = {metrics['accuracy']:.2f}%, Certified Accuracy = {metrics['certified_accuracy']:.2f}%\n")
                f.write(f"  {attack_name}: Time = {fold_timing[attack_name]:.2f} seconds\n")


        fold_results.append({
            "fold": fold + 1,
            "adversarial_accuracies": fold_adversarial_accuracies,
        })
        timing_results.append(fold_timing)

        avg_timing = {attack_name: np.mean([timing[attack_name] for timing in timing_results]) for attack_name in adversarial_loaders.keys()}

    # Calculate and print averages
    print("\n===== Adversarial Evaluation Summary =====")
    with open(output_file, "a") as f:
        f.write("\n===== Adversarial Evaluation Summary =====\n")

        for attack_name, metrics in adversarial_accuracies_summary.items():
            avg_accuracy = np.mean([m[0] for m in metrics])
            avg_certified_accuracy = np.mean([m[1] for m in metrics])

            print(f"{attack_name}: Average Accuracy = {avg_accuracy:.2f}%, Average Certified Accuracy = {avg_certified_accuracy:.2f}%")
            f.write(f"{attack_name}: Average Accuracy = {avg_accuracy:.2f}%, Average Certified Accuracy = {avg_certified_accuracy:.2f}%\n")

                # Log average timing
        print("\n===== Average Timing Summary =====")
        f.write("\n===== Average Timing Summary =====\n")
        for attack_name, avg_time in avg_timing.items():
            print(f"  {attack_name}: Average Time = {avg_time:.2f} seconds")
            f.write(f"  {attack_name}: Average Time = {avg_time:.2f} seconds\n")

    return fold_results


In [ ]:
X_test_np = X_test_tensor.cpu().numpy()
np.savetxt("X_test_clean.txt", X_test_np, delimiter=",")

In [ ]:
adversarial_loaders = {
    "PGD": "X_adv_pgd_unsw_restored_M.csv",
    "JSMA": "X_adv_jsma_unsw_restored_M.csv",
    "DeepFool": "X_adv_deepfool_unsw_restored_M.csv",
    "FGSM": "X_adv_fgsm_unsw_restored_M.csv",
    "ZOO": "X_adv_zoo_unsw_restored_M.csv",
    "CW": "X_adv_cw_unsw_restored_M.csv",
    "LPF": "X_adv_lpf_unsw_restored_M.csv",
    "HSJ": "X_adv_hsj_unsw_restored_M.csv",
}

results = train_and_evaluate_on_adversarial_folds(
    base_model_class=TabularNN,
    adaptive_model_class=Smooth, #original
    X_train_tensor=X_train_tensor,  # Training data
    y_train_tensor=y_train_tensor,  # Training labels
    X_test_tensor=X_test_tensor,    # Testing data
    y_test_tensor=y_test_tensor,    # Testing labels
    adversarial_loaders=adversarial_loaders,
    num_classes=2,
    initial_sigma=0.5,
    batch_size=64,
    num_epochs=30,
    folds=5,
    output_file="masking_less_unsw_originalSmoothing.txt",
)


Initializing models...
Training OriginalSmooth model...
Epoch 1/30, Loss: 0.3015
Epoch 2/30, Loss: 0.2689
Epoch 3/30, Loss: 0.2655
Epoch 4/30, Loss: 0.2613
Epoch 5/30, Loss: 0.2622
Epoch 6/30, Loss: 0.2628
Epoch 7/30, Loss: 0.2593
Epoch 8/30, Loss: 0.2598
Epoch 9/30, Loss: 0.2587
Epoch 10/30, Loss: 0.2585
Epoch 11/30, Loss: 0.2570
Epoch 12/30, Loss: 0.2572
Epoch 13/30, Loss: 0.2544
Epoch 14/30, Loss: 0.2582
Epoch 15/30, Loss: 0.2574
Epoch 16/30, Loss: 0.2554
Epoch 17/30, Loss: 0.2537
Epoch 18/30, Loss: 0.2557
Epoch 19/30, Loss: 0.2555
Epoch 20/30, Loss: 0.2521
Epoch 21/30, Loss: 0.2541
Epoch 22/30, Loss: 0.2549
Epoch 23/30, Loss: 0.2543
Epoch 24/30, Loss: 0.2540
Epoch 25/30, Loss: 0.2530
Epoch 26/30, Loss: 0.2546
Epoch 27/30, Loss: 0.2531
Epoch 28/30, Loss: 0.2510
Epoch 29/30, Loss: 0.2525
Epoch 30/30, Loss: 0.2529
Evaluating OriginalSmooth model on clean testing data...
Test Accuracy: 89.97%
Clean Test Accuracy: 89.97%

===== Fold 1/5 =====

Evaluating PGD adversarial examples on Fold

/tmp/ipython-input-2081315512.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_adv_sample = torch.tensor(X_adv_test[i]).unsqueeze(0).to(device)


 Accuracy on adversarial examples: 97.61%
 Certified accuracy: 94.44%

Evaluating PGDnorm2 adversarial examples on Fold 1...
 Accuracy on adversarial examples: 98.86%
 Certified accuracy: 98.83%

Evaluating FGSM adversarial examples on Fold 1...
 Accuracy on adversarial examples: 98.83%
 Certified accuracy: 98.23%

Evaluating FGSMnorm2 adversarial examples on Fold 1...
 Accuracy on adversarial examples: 98.86%
 Certified accuracy: 98.83%

Evaluating CW adversarial examples on Fold 1...
 Accuracy on adversarial examples: 95.79%
 Certified accuracy: 89.20%

Evaluating CWlessaggre adversarial examples on Fold 1...
 Accuracy on adversarial examples: 63.55%
 Certified accuracy: 62.10%

Evaluating LPF adversarial examples on Fold 1...
 Accuracy on adversarial examples: 93.12%
 Certified accuracy: 85.11%

Evaluating LPFlessaggre adversarial examples on Fold 1...
 Accuracy on adversarial examples: 93.12%
 Certified accuracy: 84.97%

===== Fold 2/5 =====

Evaluating PGD adversarial examples on 